In [ ]:
!pip install langchain langchain_openai langchain_community langgraph-checkpoint-sqlite langgraph ipykernel python-dotenv wikipedia youtube_search

Langchain community provides tool to call Wikipedia.

In [ ]:
from langchain_community.tools import WikipediaQueryRun  # pip install wikipedia
from langchain_community.utilities import WikipediaAPIWrapper

wiki_api_wrapper = WikipediaAPIWrapper(top_k_results=1, doc_content_chars_max=250)
wikipedia = WikipediaQueryRun(description="A tool to explain things in text format. Use this tool if you think the user’s asked concept is best explained through text.", api_wrapper=wiki_api_wrapper)
print(wikipedia.invoke("Explain ohm's law"))

And YouTube too!!

In [ ]:
from langchain_community.tools import YouTubeSearchTool  # pip install youtube_search

youtube = YouTubeSearchTool(description="A tool for searching YouTube videos. Use this tool when a concept is best explained visually through a video.")
youtube.run("Explain the big bang theory")

In [ ]:
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage, AIMessage, SystemMessage
from dotenv import load_dotenv

load_dotenv()

llm = ChatOpenAI(
    base_url="https://api.openai.com/v1",
    model="gpt-5.2",
    temperature=0.1,
    max_tokens=1000,
)

from langchain_classic.agents import create_tool_calling_agent, AgentExecutor
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from pprint import pprint

# Create a prompt with system message
prompt = ChatPromptTemplate.from_messages([
    ("system", """
    You are a friendly bot. Based on the user's input, select one of the tools to answer the question. Follow the rules below:
    - If user's question is related to IT, use Youtube tool.
    - Other types of question, use Wikipedia tool.
    - If user asked about past questions, use the chat history.
    - Do NOT provide addition information from your own knowledge.
    """),
    MessagesPlaceholder(variable_name="chat_history", optional=True),
    ("human", "{input}"),
    MessagesPlaceholder(variable_name="agent_scratchpad"),
])

# Define the tools list once
agent_tools = [youtube, wikipedia]

agent = create_tool_calling_agent(llm, agent_tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=agent_tools, verbose=True)

# Initialize chat history, it acts as a memory for the agent.
chat_history = []


In [ ]:
# First turn
first_input = "Explain concept of TLS handshake."
print(f"Human: {first_input}")
first_response = agent_executor.invoke({"input": first_input, "chat_history": chat_history})
print(f"AI: {first_response['output']}")

# Store first turn in chat history
chat_history.append(HumanMessage(content=first_input))
chat_history.append(AIMessage(content=first_response['output']))

print("\n--- Current Chat History ---")
pprint(chat_history)
print("----------------------------\n")

Ask another question and observe the chat history.

In [ ]:
# Second turn (follow-up question)
second_input = "Explain the Big Bang theory."
print(f"Human: {second_input}")
second_response = agent_executor.invoke({"input": second_input, "chat_history": chat_history})
print(f"AI: {second_response['output']}")

# Store second turn in chat history
chat_history.append(HumanMessage(content=second_input))
chat_history.append(AIMessage(content=second_response['output']))

print("\n--- Updated Chat History ---")
pprint(chat_history)
print("----------------------------")

With memory, such as the chat history, the agent can use it as context to answer questions.

In [ ]:
third_input = "What were the questions asked?"
print(f"Human: {third_input}")
third_response = agent_executor.invoke({"input": third_input, "chat_history": chat_history})
print(f"AI: {third_response['output']}")

Let's add a tool to the agent's toolbox. Basically a tool is just a function that enables the agent to connect to the external environment.

In [ ]:
from langchain.tools import tool

@tool
def query_ohm_law(query: str) -> str:
    """Search the customer database for records matching the query.

    Args:
        query: Search terms to look for
        limit: Maximum number of results to return
    """
    print(f"Query: {query}")
    return f"Ohm's Law states that the current (I) through a conductor is directly proportional to the voltage (V) across it and inversely proportional to the resistance (R), expressed as the formula V = I × R, meaning voltage equals current times resistance, or volts = amps times ohms."

# Define the tools list once
agent_tools = [youtube, wikipedia, query_ohm_law]

# Create a prompt with system message
prompt = ChatPromptTemplate.from_messages([
    ("system", """
    You are a friendly bot. Based on the user's input, select one of the tools to answer the question. Follow the rules below:
    - If user's question is related to IT, use Youtube tool.
    - If user's question is related to Ohm's law, use query_ohm_law tool.
    - Other types of question, use Wikipedia tool.
    - If user asked about past questions, use the chat history.
    - Do NOT provide addition information from your own knowledge.
    """),
    MessagesPlaceholder(variable_name="chat_history", optional=True),
    ("human", "{input}"),
    MessagesPlaceholder(variable_name="agent_scratchpad"),
])
agent = create_tool_calling_agent(llm, agent_tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=agent_tools, verbose=True)

fourth_input = "Tell me more about Ohm's law?"
print(f"Human: {fourth_input}")
fourth_response = agent_executor.invoke({"input": fourth_input, "chat_history": chat_history})
print(f"AI: {fourth_response['output']}")